In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam-processed-5-new/steam_subset_5_processed_new.csv
/kaggle/input/steam-20-processed-version/steam_reviews_subset_20_processed_4_25.csv


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import re

# import spacy
# spacy.prefer_gpu()
# nlp = spacy.load("en_core_web_sm")

In [3]:
# cleaned and processed 20% data sample taken from steam reviews
# data = pd.read_csv("/kaggle/input/steam-20-processed-version/steam_reviews_subset_20_processed_4_25.csv")

In [4]:
# cleaned and processed 5% data sample taken from steam reviews
data = pd.read_csv("/kaggle/input/steam-processed-5-new/steam_subset_5_processed_new.csv")

In [5]:
data.head()

,app_id,app_name,review_text,review_score,review_votes,review_processed
0,237930,Transistor,I think the main thing that surprised me about...,1,0,think main thing surprise game intigration mus...
1,270450,Robot Roller-Derby Disco Dodgeball,Cute little FPS that takes a little practice t...,1,0,cute little fps take little practice learn cat...
2,49520,Borderlands 2,This game is absolutely the best! Totally wort...,1,0,game absolutely good totally worth get cheap s...
3,211400,Deadlight,"Nope, just a nope. Clunky control, I can't sa...",0,0,nope nope clunky control frustrating play ga...
4,219640,Chivalry: Medieval Warfare,I went into Chiv expecting something amazing. ...,1,0,go chiv expect amazing exactally think go ga...


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195564 entries, 0 to 195563
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   app_id            195564 non-null  int64 
 1   app_name          195564 non-null  object
 2   review_text       195564 non-null  object
 3   review_score      195564 non-null  int64 
 4   review_votes      195564 non-null  int64 
 5   review_processed  195452 non-null  object
dtypes: int64(3), object(3)
memory usage: 9.0+ MB


In [7]:
data.isnull().sum()

app_id                0
app_name              0
review_text           0
review_score          0
review_votes          0
review_processed    112
dtype: int64

In [8]:
# data["app_name"].value_counts(normalize=True).head(20).plot(kind="bar")
# plt.xlabel("Game Name")
# plt.title("Top 20 Review Games on Steam")
# plt.show()

In [9]:
# remove rows without any data in processed review
data = data[~data["review_processed"].isnull()].copy(deep=True)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195452 entries, 0 to 195563
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   app_id            195452 non-null  int64 
 1   app_name          195452 non-null  object
 2   review_text       195452 non-null  object
 3   review_score      195452 non-null  int64 
 4   review_votes      195452 non-null  int64 
 5   review_processed  195452 non-null  object
dtypes: int64(3), object(3)
memory usage: 10.4+ MB


## Data Processing and Modeling: Imbalanced Sampling Techniques

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
# from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgbm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# from imblearn.over_sampling import SMOTE
# from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours

In [12]:
# function to show results
def show_metrics(y_test, predictions):
    print("Confusion Matrix: ")
    cmat = metrics.confusion_matrix(y_test, predictions)
    print(cmat)
    print("True Positives (TP) = ", cmat[0,0])
    print("True Negatives (TN) = ", cmat[1,1])
    print("False Positives (FP) = ", cmat[0,1])
    print("False Negatives (FN) = ", cmat[1,0])
    print("Accuracy: ")
    print(metrics.accuracy_score(y_test, predictions))
    print("Precision: ")
    print(metrics.precision_score(y_test, predictions))
    print("F1 Score: ")
    print(metrics.f1_score(y_test, predictions))
    print("Classification Report: ")
    print(metrics.classification_report(y_test, predictions))

In [13]:
X = data["review_processed"]
y = data["review_score"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [15]:
pipe = Pipeline([("cvector", CountVectorizer()), 
                 ("tfidf", TfidfTransformer())
                ])

X_train_tfvec = pipe.fit_transform(X_train)

X_test_tfvec = pipe.transform(X_test)

X_train_tfvec.shape

(156361, 91863)

In [16]:
enn = EditedNearestNeighbours()

X_train_sm, y_train_sm = enn.fit_resample(X_train_tfvec, y_train)

In [17]:
X_train_sm.shape

(62088, 91863)

## Logistic Regression

In [18]:
%%time
lr = LogisticRegression(max_iter=2000, random_state=42)
lr.fit(X_train_sm, y_train_sm)

lr_pred = lr.predict(X_test_tfvec)

print("Logistic Regression: ")
lr_train = lr.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, lr_train))

show_metrics(y_test, lr_pred)

Logistic Regression: 
Training Accuracy:  0.9238178069836361
Confusion Matrix: 
[[ 5669   909]
 [ 9569 22944]]
True Positives (TP) =  5669
True Negatives (TN) =  22944
False Positives (FP) =  909
False Negatives (FN) =  9569
Accuracy: 
0.7319587628865979
Precision: 
0.9618915859640297
F1 Score: 
0.8141077954795445
Classification Report: 
              precision    recall  f1-score   support

           0       0.37      0.86      0.52      6578
           1       0.96      0.71      0.81     32513

    accuracy                           0.73     39091
   macro avg       0.67      0.78      0.67     39091
weighted avg       0.86      0.73      0.76     39091

CPU times: user 7.33 s, sys: 6.67 s, total: 14 s
Wall time: 4.16 s


In [19]:
%%time
# tuning with gridsearch
param_grid = {"C": [0.01, 0.1, 1]}

lr_cv = GridSearchCV(lr, param_grid, cv=5)
lr_cv.fit(X_train_sm, y_train_sm)

print("Best Params: ", lr_cv.best_params_)
print("Best Score: ", lr_cv.best_score_)

Best Params:  {'C': 1}
Best Score:  0.9019294992643891
CPU times: user 56 s, sys: 54.2 s, total: 1min 50s
Wall time: 30.4 s


In [20]:
new_lr = LogisticRegression(max_iter=2000, random_state=42, **lr_cv.best_params_)
new_lr.fit(X_train_sm, y_train_sm)

lr_pred = new_lr.predict(X_test_tfvec)

lr_train = new_lr.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, lr_train))

show_metrics(y_test, lr_pred)

Training Accuracy:  0.9238178069836361
Confusion Matrix: 
[[ 5669   909]
 [ 9569 22944]]
True Positives (TP) =  5669
True Negatives (TN) =  22944
False Positives (FP) =  909
False Negatives (FN) =  9569
Accuracy: 
0.7319587628865979
Precision: 
0.9618915859640297
F1 Score: 
0.8141077954795445
Classification Report: 
              precision    recall  f1-score   support

           0       0.37      0.86      0.52      6578
           1       0.96      0.71      0.81     32513

    accuracy                           0.73     39091
   macro avg       0.67      0.78      0.67     39091
weighted avg       0.86      0.73      0.76     39091



## Decision Tree

In [21]:
%%time
dtc_clf = DecisionTreeClassifier(random_state=42)
dtc_clf.fit(X_train_sm, y_train_sm)

dtc_pred = dtc_clf.predict(X_test_tfvec)

print("Decision Tree: ")
dtc_train = dtc_clf.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, dtc_train))

show_metrics(y_test, dtc_pred)

Decision Tree: 
Training Accuracy:  0.9985021260146888
Confusion Matrix: 
[[ 5148  1430]
 [13221 19292]]
True Positives (TP) =  5148
True Negatives (TN) =  19292
False Positives (FP) =  1430
False Negatives (FN) =  13221
Accuracy: 
0.625207848353841
Precision: 
0.93099121706399
F1 Score: 
0.7247863247863249
Classification Report: 
              precision    recall  f1-score   support

           0       0.28      0.78      0.41      6578
           1       0.93      0.59      0.72     32513

    accuracy                           0.63     39091
   macro avg       0.61      0.69      0.57     39091
weighted avg       0.82      0.63      0.67     39091

CPU times: user 52.7 s, sys: 10.2 ms, total: 52.7 s
Wall time: 52.7 s


In [22]:
%%time
# tuning with gridsearch
param_grid = {"max_depth": [2, 5, 8, 12, 18], 
              "min_samples_leaf": [5, 10, 20, 35, 60], 
              "criterion": ["gini", "entropy"]}

dtc_cv = GridSearchCV(dtc_clf, param_grid, cv=5)
dtc_cv.fit(X_train_sm, y_train_sm)

print("Best Params: ", dtc_cv.best_params_)
print("Best Score: ", dtc_cv.best_score_)

Best Params:  {'criterion': 'gini', 'max_depth': 18, 'min_samples_leaf': 35}
Best Score:  0.7934544703615709
CPU times: user 13min 6s, sys: 148 ms, total: 13min 6s
Wall time: 13min 6s


In [23]:
new_dtc = DecisionTreeClassifier(random_state=42, **dtc_cv.best_params_)
new_dtc.fit(X_train_sm, y_train_sm)

dtc_pred = new_dtc.predict(X_test_tfvec)

dtc_train = new_dtc.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, dtc_train))

show_metrics(y_test, dtc_pred)

Training Accuracy:  0.799268779796418
Confusion Matrix: 
[[ 4413  2165]
 [11765 20748]]
True Positives (TP) =  4413
True Negatives (TN) =  20748
False Positives (FP) =  2165
False Negatives (FN) =  11765
Accuracy: 
0.6436519915069965
Precision: 
0.9055121546720203
F1 Score: 
0.7486739075524123
Classification Report: 
              precision    recall  f1-score   support

           0       0.27      0.67      0.39      6578
           1       0.91      0.64      0.75     32513

    accuracy                           0.64     39091
   macro avg       0.59      0.65      0.57     39091
weighted avg       0.80      0.64      0.69     39091



## Random Forest

In [24]:
%%time
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_sm, y_train_sm)

rf_pred = rf_clf.predict(X_test_tfvec)

print("Random Forest: ")
rf_train = rf_clf.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, rf_train))

show_metrics(y_test, rf_pred)

Random Forest: 
Training Accuracy:  0.9984699136709186
Confusion Matrix: 
[[ 5607   971]
 [13495 19018]]
True Positives (TP) =  5607
True Negatives (TN) =  19018
False Positives (FP) =  971
False Negatives (FN) =  13495
Accuracy: 
0.6299403954874524
Precision: 
0.951423282805543
F1 Score: 
0.7244676393280256
Classification Report: 
              precision    recall  f1-score   support

           0       0.29      0.85      0.44      6578
           1       0.95      0.58      0.72     32513

    accuracy                           0.63     39091
   macro avg       0.62      0.72      0.58     39091
weighted avg       0.84      0.63      0.68     39091

CPU times: user 9min 29s, sys: 534 ms, total: 9min 29s
Wall time: 9min 29s


In [25]:
%%time
# tuning with gridsearch
param_grid = {"max_depth": [2, 5, 8, 12, 18], 
              "min_samples_leaf": [5, 10, 20, 35, 60], 
              "n_estimators": [30, 50, 80, 150]}

rf_cv = GridSearchCV(rf_clf, param_grid, cv=5)
rf_cv.fit(X_train_sm, y_train_sm)

print("Best Params: ", rf_cv.best_params_)
print("Best Score: ", rf_cv.best_score_)

Best Params:  {'max_depth': 18, 'min_samples_leaf': 20, 'n_estimators': 30}
Best Score:  0.7415443784286043
CPU times: user 31min, sys: 1.56 s, total: 31min 1s
Wall time: 31min 1s


In [26]:
new_rf = RandomForestClassifier(random_state=42, **rf_cv.best_params_)
new_rf.fit(X_train_sm, y_train_sm)

rf_pred = new_rf.predict(X_test_tfvec)

rf_train = new_rf.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, rf_train))

show_metrics(y_test, rf_pred)

Training Accuracy:  0.7411093931194433
Confusion Matrix: 
[[ 2706  3872]
 [ 6507 26006]]
True Positives (TP) =  2706
True Negatives (TN) =  26006
False Positives (FP) =  3872
False Negatives (FN) =  6507
Accuracy: 
0.7344913151364765
Precision: 
0.870406319030725
F1 Score: 
0.8336458784119505
Classification Report: 
              precision    recall  f1-score   support

           0       0.29      0.41      0.34      6578
           1       0.87      0.80      0.83     32513

    accuracy                           0.73     39091
   macro avg       0.58      0.61      0.59     39091
weighted avg       0.77      0.73      0.75     39091



## XGBoost

In [27]:
%%time
xgb_clf = XGBClassifier(random_state=42)
xgb_clf.fit(X_train_sm, y_train_sm)

xgb_pred = xgb_clf.predict(X_test_tfvec)

print("XGBoost: ")
xgb_train = xgb_clf.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, xgb_train))

show_metrics(y_test, xgb_pred)

XGBoost: 
Training Accuracy:  0.8971620925138513
Confusion Matrix: 
[[ 5354  1224]
 [11316 21197]]
True Positives (TP) =  5354
True Negatives (TN) =  21197
False Positives (FP) =  1224
False Negatives (FN) =  11316
Accuracy: 
0.6792100483487248
Precision: 
0.9454083225547478
F1 Score: 
0.7717260712855426
Classification Report: 
              precision    recall  f1-score   support

           0       0.32      0.81      0.46      6578
           1       0.95      0.65      0.77     32513

    accuracy                           0.68     39091
   macro avg       0.63      0.73      0.62     39091
weighted avg       0.84      0.68      0.72     39091

CPU times: user 51 s, sys: 404 ms, total: 51.4 s
Wall time: 51.4 s


## Naive Bayes

In [28]:
%%time
clf = MultinomialNB()
clf.fit(X_train_sm, y_train_sm)

nb_pred = clf.predict(X_test_tfvec)

print("Naive Bayes: ")
nb_train = clf.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, nb_train))

show_metrics(y_test, nb_pred)

Naive Bayes: 
Training Accuracy:  0.9184866640896792
Confusion Matrix: 
[[ 5762   816]
 [12484 20029]]
True Positives (TP) =  5762
True Negatives (TN) =  20029
False Positives (FP) =  816
False Negatives (FN) =  12484
Accuracy: 
0.6597682330971323
Precision: 
0.9608539218037899
F1 Score: 
0.7507402826192887
Classification Report: 
              precision    recall  f1-score   support

           0       0.32      0.88      0.46      6578
           1       0.96      0.62      0.75     32513

    accuracy                           0.66     39091
   macro avg       0.64      0.75      0.61     39091
weighted avg       0.85      0.66      0.70     39091

CPU times: user 165 ms, sys: 996 µs, total: 166 ms
Wall time: 164 ms


## K Nearest Neighbors

In [29]:
# %%time
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train_tfvec, y_train)

# knn_pred = knn.predict(X_test_tfvec)

# show_metrics(y_test, knn_pred)

In [30]:
# %%time
# # tuning with gridsearch
# param_grid = {"n_neighbors" : [3, 4, 5, 6, 7, 8]}

# knn_cv = GridSearchCV(knn, param_grid, cv=5)
# knn_cv.fit(X_train_tfvec, y_train)

# print("Best Params: ", knn_cv.best_params_)
# print("Best Score: ", knn_cv.best_score_)

## LightGBM

In [31]:
# %%time
# lgb_clf = lgbm.LGBMClassifier(random_state=42)
# lgb_clf.fit(X_train_sm, y_train_sm)

# lgb_pred = lgb_clf.predict(X_test_tfvec)

# lgb_train = lgb_clf.predict(X_train_sm)
# print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, lgb_train))

# show_metrics(y_test, lgb_pred)

In [32]:
# %%time
# # tuning with gridsearch
# param_grid = {"n_estimators": [50, 100, 150, 200], 
#               "learning_rate": [0.001, 0.01, 0.1, 1], 
#               "boosting_type": ["gbdt", "dart", "rf"]}

# lgb_cv = GridSearchCV(lgb_clf, param_grid, cv=5)
# lgb_cv.fit(X_train_sm, y_train_sm)

# print("Best Params: ", lgb_cv.best_params_)
# print("Best Score: ", lgb_cv.best_score_)

## AdaBoost

In [33]:
%%time
ada = AdaBoostClassifier(random_state=42)
ada.fit(X_train_sm, y_train_sm)

ada_pred = ada.predict(X_test_tfvec)

print("AdaBoost: ")
ada_train = ada.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, ada_train))

show_metrics(y_test, ada_pred)

AdaBoost: 
Training Accuracy:  0.8331561654425976
Confusion Matrix: 
[[ 4476  2102]
 [ 8470 24043]]
True Positives (TP) =  4476
True Negatives (TN) =  24043
False Positives (FP) =  2102
False Negatives (FN) =  8470
Accuracy: 
0.7295541173160063
Precision: 
0.9196022183973991
F1 Score: 
0.819768829486174
Classification Report: 
              precision    recall  f1-score   support

           0       0.35      0.68      0.46      6578
           1       0.92      0.74      0.82     32513

    accuracy                           0.73     39091
   macro avg       0.63      0.71      0.64     39091
weighted avg       0.82      0.73      0.76     39091

CPU times: user 1min 28s, sys: 14.3 ms, total: 1min 28s
Wall time: 1min 28s


In [34]:
%%time
# tuning with gridsearch
param_grid = {"n_estimators" : [50, 70, 90, 110, 140, 180], 
              "learning_rate" : [0.001, 0.01, 0.1, 1, 10]}

ada_cv = GridSearchCV(ada, param_grid, cv=5)
ada_cv.fit(X_train_sm, y_train_sm)

print("Best Params: ", ada_cv.best_params_)
print("Best Score: ", ada_cv.best_score_)

Best Params:  {'learning_rate': 1, 'n_estimators': 180}
Best Score:  0.8691212748154268
CPU times: user 5h 34min 53s, sys: 3.78 s, total: 5h 34min 57s
Wall time: 5h 34min 58s


In [35]:
new_ada = AdaBoostClassifier(random_state=42, **ada_cv.best_params_)
new_ada.fit(X_train_sm, y_train_sm)

ada_pred = new_ada.predict(X_test_tfvec)

ada_train = new_ada.predict(X_train_sm)
print("Training Accuracy: ", metrics.accuracy_score(y_train_sm, ada_train))

show_metrics(y_test, ada_pred)

Training Accuracy:  0.8743879654683675
Confusion Matrix: 
[[ 5193  1385]
 [ 9971 22542]]
True Positives (TP) =  5193
True Negatives (TN) =  22542
False Positives (FP) =  1385
False Negatives (FN) =  9971
Accuracy: 
0.7094983500038372
Precision: 
0.942115601621599
F1 Score: 
0.7987951807228915
Classification Report: 
              precision    recall  f1-score   support

           0       0.34      0.79      0.48      6578
           1       0.94      0.69      0.80     32513

    accuracy                           0.71     39091
   macro avg       0.64      0.74      0.64     39091
weighted avg       0.84      0.71      0.74     39091



In [36]:
# %%time
# # use svc as a base learner
# svc = SVC(probability=True, kernel="linear", random_state=42)

# ada_svc = AdaBoostClassifier(base_estimator=svc, random_state=42)
# ada_svc.fit(X_train_tfvec, y_train)

# ada_svc_pred = ada_svc.predict(X_test_tfvec)

# show_metrics(y_test, ada_svc_pred)